In [1]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
from PIL import Image

# Mediapipe 얼굴 랜드마크 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# EXIF 회전 보정 함수
def load_image_with_exif_orientation(image_path):
    image = Image.open(image_path)
    image = image.convert("RGB")
    
    try:
        exif = image._getexif()
        if exif is not None:
            for orientation in ExifTags.TAGS.keys():
                if ExifTags.TAGS[orientation] == 'Orientation':
                    break
            if orientation in exif:
                orientation_value = exif[orientation]
                if orientation_value == 3:
                    image = image.rotate(180, expand=True)
                elif orientation_value == 6:
                    image = image.rotate(270, expand=True)
                elif orientation_value == 8:
                    image = image.rotate(90, expand=True)
    except:
        pass

    return np.array(image)

# 이미지 리사이즈 함수
def resize_image(image, max_width=640):
    h, w = image.shape[:2]
    if w > max_width:
        scale = max_width / w
        image = cv2.resize(image, (int(w * scale), int(h * scale)))
    return image

# 얼굴 랜드마크 추출 함수
def extract_face_landmarks(image_path, normalize=True):
    image = load_image_with_exif_orientation(image_path)
    image = resize_image(image)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    results = face_mesh.process(image_rgb)

    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0]
        coords = np.array([[lm.x, lm.y] for lm in landmarks.landmark])
        
        if normalize:
            coords -= np.mean(coords, axis=0)
        
        return coords
    
    return None  # 얼굴을 찾지 못한 경우

I0000 00:00:1746154325.602326   55690 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1746154325.610660   56523 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746154325.612940   56528 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [6]:
# 모델 불러오기
model = load_model("concentration_model.h5")  # 저장된 모델 경로로 변경

# 감정 클래스 라벨 정의
emotion_labels = {0: "Focused", 1: "Not Focused", 2: "Drowsy"}

# 이미지 테스트 함수
def test_image(image_path):
    # 얼굴 랜드마크 추출
    landmarks = extract_face_landmarks(image_path)
    
    if landmarks is not None:
        # 모델에 맞게 데이터 차원 변경
        landmarks = np.expand_dims(landmarks, axis=0)  # (1, 468, 2)로 변경
        
        # 예측
        prediction = model.predict(landmarks)
        
        # 예측된 라벨을 감정 라벨로 변환
        predicted_class = np.argmax(prediction, axis=1)
        predicted_emotion = emotion_labels[predicted_class[0]]  # 감정 클래스 라벨을 변환

        print(f"Predicted emotion: {predicted_emotion}")  # 예측된 감정 출력

    else:
        print("No face detected in the image.")

# 테스트할 이미지 경로
image_path = "/Users/kimjohyeon/Desktop/Capstone/Sample2.jpg"  # 테스트할 이미지 경로로 변경

# 테스트 이미지로 예측 수행
test_image(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicted emotion: Focused
